In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from plotnine import *

In [ ]:
results_annotated = pd.read_json("../data/results_annotated.jsonl", lines=True)

results = (
    pd
    .read_json("../data/results.jsonl", lines=True)
    .merge(results_annotated[['poem','label']], how="left", on="poem")
)

results['label'].isna().sum(), results_annotated['label'].isna().sum()

In [ ]:
from abba_eval import pigeonxt
from IPython.display import display
from ipywidgets import HTML

annotations = pigeonxt.annotate(
    results.loc[results['label'].isna(),['poem']],
    example_column="poem",
    options=['Correct', 'Incorrect', 'Doubtful'],
    display_fn=lambda html: display(HTML(html.value.replace("\n", "<br>")))
)

In [ ]:
results_annotated = (
    results
    .merge(annotations[['poem','label']], how="left", on="poem", suffixes=('_x', '_y'))
    .assign(label=lambda x: x['label_y'].combine_first(x['label_x']))
    .drop(columns=['label_x', 'label_y'])
)

results_annotated.to_json("../data/results_annotated.jsonl", lines=True, orient='records')